In [1]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

Warning message:
“package ‘tidyverse’ was built under R version 4.1.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.0 
✔ readr   2.1.3      ✔ forcats 0.5.1 
Warning message:
“package ‘ggplot2’ was built under R version 4.1.2”
Warning message:
“package ‘tibble’ was built under R version 4.1.2”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidym

In [3]:
metrics <- read_csv("data/RawMetrics.csv")
head(metrics)

New names:
• `` -> `...21`
Rows: 539 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): last_name, first_name
dbl (18): player_id, year, xslg, xwoba, xobp, exit_velocity_avg, launch_angl...
lgl  (1): ...21

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


last_name,first_name,player_id,year,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,⋯,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed,...21
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Pujols,Albert,405395,2019,0.428,0.319,0.314,88.3,12.5,26.5,⋯,3.7,36.6,59.6,30.8,46.2,26.7,16.9,10.2,22.5,NA
Cabrera,Miguel,408234,2019,0.436,0.331,0.337,90.3,12.3,38.2,⋯,7.2,44.7,67.9,33.9,44.9,22.3,27.9,4.9,23.5,NA
Choo,Shin-Soo,425783,2019,0.474,0.362,0.370,91.9,9.2,31.8,⋯,9.0,49.0,64.8,20.4,50.6,22.3,23.6,3.5,26.6,NA
Jones,Adam,430945,2019,0.432,0.319,0.312,86.2,13.5,31.3,⋯,5.4,31.0,72.3,39.3,43.9,25.8,21.2,9.0,26.9,NA
Cruz Jr.,Nelson,443558,2019,0.637,0.415,0.377,93.7,13.1,38.3,⋯,8.3,51.5,69.9,27.3,40.5,24.8,27.0,7.7,25.3,NA
Longoria,Evan,446334,2019,0.491,0.351,0.340,89.7,14.0,35.8,⋯,8.4,40.8,64.6,30.1,41.0,26.6,27.2,5.2,26.4,NA


In [5]:
selected <- metrics %>% select(-"player_id", -"...21")
head(selected)

last_name,first_name,year,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pujols,Albert,2019,0.428,0.319,0.314,88.3,12.5,26.5,5.8,3.7,36.6,59.6,30.8,46.2,26.7,16.9,10.2,22.5
Cabrera,Miguel,2019,0.436,0.331,0.337,90.3,12.3,38.2,6.4,7.2,44.7,67.9,33.9,44.9,22.3,27.9,4.9,23.5
Choo,Shin-Soo,2019,0.474,0.362,0.370,91.9,9.2,31.8,8.8,9.0,49.0,64.8,20.4,50.6,22.3,23.6,3.5,26.6
Jones,Adam,2019,0.432,0.319,0.312,86.2,13.5,31.3,6.7,5.4,31.0,72.3,39.3,43.9,25.8,21.2,9.0,26.9
Cruz Jr.,Nelson,2019,0.637,0.415,0.377,93.7,13.1,38.3,19.9,8.3,51.5,69.9,27.3,40.5,24.8,27.0,7.7,25.3
Longoria,Evan,2019,0.491,0.351,0.340,89.7,14.0,35.8,7.5,8.4,40.8,64.6,30.1,41.0,26.6,27.2,5.2,26.4


In [7]:
names <- selected %>% unite("player", last_name:first_name, sep = ", ", remove = T)
head(names)

player,year,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Pujols, Albert",2019,0.428,0.319,0.314,88.3,12.5,26.5,5.8,3.7,36.6,59.6,30.8,46.2,26.7,16.9,10.2,22.5
"Cabrera, Miguel",2019,0.436,0.331,0.337,90.3,12.3,38.2,6.4,7.2,44.7,67.9,33.9,44.9,22.3,27.9,4.9,23.5
"Choo, Shin-Soo",2019,0.474,0.362,0.370,91.9,9.2,31.8,8.8,9.0,49.0,64.8,20.4,50.6,22.3,23.6,3.5,26.6
"Jones, Adam",2019,0.432,0.319,0.312,86.2,13.5,31.3,6.7,5.4,31.0,72.3,39.3,43.9,25.8,21.2,9.0,26.9
"Cruz Jr., Nelson",2019,0.637,0.415,0.377,93.7,13.1,38.3,19.9,8.3,51.5,69.9,27.3,40.5,24.8,27.0,7.7,25.3
"Longoria, Evan",2019,0.491,0.351,0.340,89.7,14.0,35.8,7.5,8.4,40.8,64.6,30.1,41.0,26.6,27.2,5.2,26.4


In [23]:
fpoints <- read_csv("data/fpoints_data.csv")
head(fpoints)

Rows: 539 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): player
dbl (10): year, K, BB, RBI, TB, SB, G, R, Fpoints, fpoints_g

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


player,year,K,BB,RBI,TB,SB,G,R,Fpoints,fpoints_g
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Soto, Juan",2020,28,41,37,107,5,47,39,201,4.276596
"Freeman, Freddie",2020,37,45,53,137,2,60,51,251,4.183333
"Yelich, Christian",2019,118,80,97,328,30,130,100,517,3.976923
"Rendon, Anthony",2019,86,80,126,326,5,146,117,568,3.890411
"Judge, Aaron",2022,175,111,131,391,16,157,133,607,3.866242
"Trout, Mike",2019,120,110,104,303,11,134,110,518,3.865672


In [25]:
fpoints_main <- fpoints %>% select("player", "year", "fpoints_g")
head(fpoints_main)

player,year,fpoints_g
<chr>,<dbl>,<dbl>
"Soto, Juan",2020,4.276596
"Freeman, Freddie",2020,4.183333
"Yelich, Christian",2019,3.976923
"Rendon, Anthony",2019,3.890411
"Judge, Aaron",2022,3.866242
"Trout, Mike",2019,3.865672


In [26]:
fpoints_grouped <- fpoints %>% group_by(player) %>% summarize(avg_fpoints = mean(fpoints_g))
head(fpoints_grouped)

player,avg_fpoints
<chr>,<dbl>
"Abreu, Jose",2.777493
"Acuna Jr., Ronald",2.891475
"Adames, Willy",1.927085
"Aguilar, Jesus",2.096501
"Ahmed, Nick",2.110759
"Alberto, Hanser",2.213896


In [27]:
colnames(names)

[1] "player"               "year"                 "xslg"                
 [4] "xwoba"                "xobp"                 "exit_velocity_avg"   
 [7] "launch_angle_avg"     "sweet_spot_percent"   "barrel_batted_rate"  
[10] "solidcontact_percent" "hard_hit_percent"     "z_swing_percent"     
[13] "oz_swing_percent"     "groundballs_percent"  "flyballs_percent"    
[16] "linedrives_percent"   "popups_percent"       "sprint_speed"

In [29]:
met <- names %>% group_by(player) %>% summarize(across(everything(), mean)) %>% select(-year)
head(met)

player,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Abreu, Jose",0.5117500,0.3697500,0.3520000,92.30000,10.05000,34.57500,11.700000,8.075000,50.65000,67.000,32.25000,46.55000,23.72500,25.00000,4.700000,26.47500
"Acuna Jr., Ronald",0.5503333,0.3920000,0.3773333,91.40000,14.53333,35.76667,14.600000,11.066667,51.36667,68.700,22.76667,40.73333,28.70000,24.16667,6.400000,29.03333
"Adames, Willy",0.4300000,0.3187500,0.3077500,88.92500,14.75000,36.12500,10.675000,7.550000,41.10000,70.425,27.07500,40.92500,27.20000,24.37500,7.500000,28.07500
"Aguilar, Jesus",0.4326667,0.3256667,0.3216667,88.86667,18.03333,37.30000,7.533333,6.366667,38.16667,64.100,31.86667,34.56667,28.33333,28.60000,8.633333,24.23333
"Ahmed, Nick",0.3995000,0.3125000,0.3160000,87.75000,8.50000,32.90000,5.900000,5.350000,33.65000,70.800,29.10000,50.05000,18.90000,25.20000,5.800000,28.15000
"Alberto, Hanser",0.3555000,0.2805000,0.2885000,83.35000,12.55000,32.55000,1.950000,2.900000,19.60000,75.100,48.80000,47.65000,21.05000,23.65000,7.600000,26.85000


In [30]:
full_data <- right_join(met, fpoints_grouped, by = "player")
head(full_data)

player,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed,avg_fpoints
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Abreu, Jose",0.5117500,0.3697500,0.3520000,92.30000,10.05000,34.57500,11.700000,8.075000,50.65000,67.000,32.25000,46.55000,23.72500,25.00000,4.700000,26.47500,2.777493
"Acuna Jr., Ronald",0.5503333,0.3920000,0.3773333,91.40000,14.53333,35.76667,14.600000,11.066667,51.36667,68.700,22.76667,40.73333,28.70000,24.16667,6.400000,29.03333,2.891475
"Adames, Willy",0.4300000,0.3187500,0.3077500,88.92500,14.75000,36.12500,10.675000,7.550000,41.10000,70.425,27.07500,40.92500,27.20000,24.37500,7.500000,28.07500,1.927085
"Aguilar, Jesus",0.4326667,0.3256667,0.3216667,88.86667,18.03333,37.30000,7.533333,6.366667,38.16667,64.100,31.86667,34.56667,28.33333,28.60000,8.633333,24.23333,2.096501
"Ahmed, Nick",0.3995000,0.3125000,0.3160000,87.75000,8.50000,32.90000,5.900000,5.350000,33.65000,70.800,29.10000,50.05000,18.90000,25.20000,5.800000,28.15000,2.110759
"Alberto, Hanser",0.3555000,0.2805000,0.2885000,83.35000,12.55000,32.55000,1.950000,2.900000,19.60000,75.100,48.80000,47.65000,21.05000,23.65000,7.600000,26.85000,2.213896


In [31]:
write_csv(full_data, "data/full_data.csv")